In [2]:
import requests
import pandas as pd
import numpy as np
import math
from bs4 import BeautifulSoup
from datetime import datetime

In [3]:
url= 'https://www2.hm.com/en_us/productpage.0985159005.html'

headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36 OPR/37.0.2178.54'}

page = requests.get( url, headers=headers )

In [4]:
# Just One Products

#Beautifull Souo object
soup= BeautifulSoup(page.text,'html.parser')

### ===================== color name ======================

products_list= soup.find_all('a',class_='filter-option miniature')

#product id
products_id = [p.get('data-articlecode') for p in products_list]

#color name
color_name = [p.get('data-color') for p in products_list]


#create data frame

df_color = pd.DataFrame([products_id,color_name]).T

#rename the columns of DF
df_color.columns= ['product_id','color_name']

#create new colum styly_id
df_color['stlye_id'] = df_color['product_id'].apply(lambda x: x[:-3])

#create new colum color_id
df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])

# ============================== composition =============================== #

#composition
composition_list= soup.find_all('div', class_ = 'details-attributes-list-item')

product_composition = [list(filter (None, p.get_text().split('\n') ) ) for p in composition_list]

#created the dataframe aux
df_aux= pd.DataFrame(product_composition).T
#renome the columns
df_aux.columns= df_aux.iloc[0]

#select columns 
df_aux= df_aux[['Fit','Composition','Art. No.']]

#delete the first row and delete the columns that is the empty
df_aux= df_aux.iloc[1:3]

#copying the code and the model to row the below
df_aux = df_aux.fillna(method='ffill')

#rename the data frame to a new variable
df_composition = df_aux.copy(True)

#create new colum styly_id
df_composition['stlye_id'] =df_composition['Art. No.'].apply(lambda x: x[:-3])

#create new colum color_id
df_composition['color_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])

#merge dataframe of color and composition 
data_sku= pd.merge(df_color,df_composition[['stlye_id','Fit','Composition']], how='left', on ='stlye_id')